In [1]:
import numpy as np

In [201]:
Id = lambda x: x
MESSAGE = "Starting point outside of the domain."

In [226]:
def indomain(x, domain):
    return np.alltrue(domain[:, 0] < x) and np.alltrue(x < domain[:, 1])


def move_elementwise(x, law, domain=None, width=1, seed=None, **kvargs):
    x = np.array(x)
    d = len(x)
    
    if domain is None:
        domain = np.ones((d, 2))
        domain[:, 0] = -np.inf * domain[:, 0]
        domain[:, 1] = np.inf * domain[:, 1]
    domain = np.array(domain)
    
    if np.isscalar(width):
        width = width * np.ones(d)
        
    if seed is not None:
        np.random.seed(seed)
    
    if not indomain(x, domain):
        raise Exception(MESSAGE)
    
    if law == 'u':
        y = x + width * (np.random.rand(d) - 0.5)
    elif law == 'n':
        y = x + width * np.random.randn(d)
        
    index = (y < domain[:, 0]) | (y > domain[:, 1])
    y[index] = x[index]
    
    return y


def move_cohert(x, domain=None, cov=1, seed=None, **kvargs):
    x = np.array(x)
    d = len(x)
    
    if domain is None:
        domain = np.ones((d, 2))
        domain[:, 0] = -np.inf * domain[:, 0]
        domain[:, 1] = np.inf * domain[:, 1]
    domain = np.array(domain)
            
    if np.isscalar(cov):
        cov = np.diag(cov*np.ones(d))

    if seed is not None:
        np.random.seed(seed)
    
    if not indomain(x, domain):
        raise Exception(MESSAGE)
        
    y = x + np.random.multivariate_normal(np.zeros(d), cov)

    if not indomain(y, domain):
        y = x
    
    return y


def mh(x, proba, domain=None, N=500, B=200, move='n', ascdes=(Id, Id), seed=None, **kvargs):
    x = np.array(x)
    d = len(x)
    
    if domain is None:
        domain = np.ones((d, 2))
        domain[:, 0] = -np.inf * domain[:, 0]
        domain[:, 1] = np.inf * domain[:, 1]
    
    if not indomain(x, domain):
        raise Exception(MESSAGE)
    
    if seed is not None:
        np.random.seed(seed)
        
    walker = np.zeros((N+1, d))
    walker[0, :] = x
    
    px = proba(x)

    _x = ascdes[0](x)
    _domain = ascdes[0](domain)
    
    for i in range(N):
        if move in ['u', 'n']:
            _y = move_elementwise(_x, move, _domain, **kvargs)
        elif move == 'c':
            _y = move_cohert(_x, _domain, **kvargs)
        else:
            _y = move(_x, _domain, **kvargs)
        
        y = ascdes[1](_y)
        py = proba(y)
        if np.random.rand() < py / px:
            _x, x, px = _y, y, py
        
        walker[i+1, :] = x
        
    return np.mean(walker[B:, :], axis=0), walker

In [263]:
zeros = np.zeros(3)
domain = np.array([[-1, 1], [-1, 1], [-1, 1]])


def test_move_elementwise():
    assert np.allclose(move_elementwise(zeros, 'u', domain, 3, 0), np.array([0.14644051, 0.6455681 , 0.30829013]))
    assert np.allclose(move_elementwise(zeros, 'u', domain, 3, 1), np.array([-0.24893399,  0.66097348,  0.]))
    assert np.allclose(move_elementwise(zeros, 'n', domain, 1, 0), np.array([0., 0.40015721, 0.97873798]))
    assert np.allclose(move_elementwise(zeros, 'n', domain, 1, 1), np.array([0., -0.61175641, -0.52817175]))
    assert np.allclose(move_elementwise([0], 'u', None, 1, 0), np.array([0.0488135]))
    

def test_move_cohert():
    assert np.allclose(move_cohert(zeros, domain, 1, 4), np.array([ 0.05056171,  0.49995133, -0.99590893]))
    
    
def test_mh():
    mh(np.zeros(2), lambda x: np.exp(-np.dot(x, x)), N=10000)
    assert np.abs(res[0]) < 0.1 and np.abs(res[1]) < 0.1
    mh(np.zeros(1), lambda x: 1, move=lambda x, d, **kv: x+1)
    
    
test_move_elementwise()
test_move_cohert()
test_mh()

In [253]:
res, walker = mh(np.zeros(2), lambda x: 1, N=10000)
fig = px.line(x=walker[:, 0], y=walker[:, 1])
fig.update_layout(
    margin=dict(b=0, l=0, r=0, t=25),
    xaxis=dict(scaleanchor="y", scaleratio=1, constrain="domain")
)

In [254]:
res

array([-110.79733209,   42.29171025])

In [255]:
res, walker = mh(np.zeros(2), lambda x: np.exp(-np.dot(x, x)), N=10000)
fig = px.scatter(x=walker[:, 0], y=walker[:, 1], opacity=0.2)
fig.update_layout(
    margin=dict(b=0, l=0, r=0, t=25),
    xaxis=dict(scaleanchor="y", scaleratio=1, constrain="domain")
)

In [256]:
res

array([-0.03120238, -0.0049903 ])

In [217]:
import plotly.graph_objects as go 
import plotly.express as px 
import plotly.io as pio
from plotly.subplots import make_subplots


In [262]:
mh(np.zeros(1), lambda x: 1, move=lambda x, d, **kv: x+1)

(array([350.]), array([[  0.],
        [  1.],
        [  2.],
        [  3.],
        [  4.],
        [  5.],
        [  6.],
        [  7.],
        [  8.],
        [  9.],
        [ 10.],
        [ 11.],
        [ 12.],
        [ 13.],
        [ 14.],
        [ 15.],
        [ 16.],
        [ 17.],
        [ 18.],
        [ 19.],
        [ 20.],
        [ 21.],
        [ 22.],
        [ 23.],
        [ 24.],
        [ 25.],
        [ 26.],
        [ 27.],
        [ 28.],
        [ 29.],
        [ 30.],
        [ 31.],
        [ 32.],
        [ 33.],
        [ 34.],
        [ 35.],
        [ 36.],
        [ 37.],
        [ 38.],
        [ 39.],
        [ 40.],
        [ 41.],
        [ 42.],
        [ 43.],
        [ 44.],
        [ 45.],
        [ 46.],
        [ 47.],
        [ 48.],
        [ 49.],
        [ 50.],
        [ 51.],
        [ 52.],
        [ 53.],
        [ 54.],
        [ 55.],
        [ 56.],
        [ 57.],
        [ 58.],
        [ 59.],
        [ 60.],
        [